In [1]:
import numpy as np

In [2]:
def build_adjM(vertexNum:int, edges:list[tuple[int, int, int]], is_directed=False)->np.ndarray:

    '''
    构建邻接矩阵. 参数:
    vertexNum: 节点个数.
    edges:边权重信息. (u, v, w)表示 u 到 v 的边权重 w.
    is_directed: 判断是否是有向图. 如果不是有向图, 则(u, v, w)表示 (u, v)之间有一个双向边, 权重都是 w.
    '''
    adjM=np.full(shape=(vertexNum, vertexNum), fill_value=np.inf, dtype=np.float32)

    if not is_directed: # 无向图
        for u, v, w in edges:
            adjM[u][v]=adjM[v][u]=w
    else: # 有向图
        for u, v, w in edges:
            adjM[u][v]=w

    return adjM


In [ ]:
def dijkstra(adjM:np.ndarray, start_id:int, with_trace_back=False)->np.ndarray:

    '''
    dijkstra 算法. 输出从起点 start_id 到图中其余各个顶点的最短距离.

    adjM: 图的邻接矩阵. 具体要求:
          - 图中所有边权值不能是非负数.
          - 图中任意两点之间最多一条边. 

    start_id: 起点.

    with_trace_back: 是否记录并且保存start_id 到每个结点的最短路径的路径轨迹. Tre 表示记录.

    输出: dist[], np.ndarray数组, dist[i]=d 表示 start_id 到下标为 i 的结点图上最短距离 d.
          d>=0, 也可能 d=np.inf.

    时间复杂度:
        两层 for 循环, O(verNum^2).
    '''

    verNum=adjM.shape[0]
    dist=np.full(shape=verNum, fill_value=np.inf, dtype=np.float32)

    visited=np.zeros(shape=verNum, dtype=np.int8)
    pred=[-1 for _ in range(verNum)]

    dist[start_id]=0
    # visited[start_id]=1 # 对 start_id 的visited会被放到下面的循环中统计
    # pred[start_id]=-1 已经给所有 pred 赋值为-1, 没必要单独赋值

    for _ in range(verNum): # verNum-1 or verNum?
        min_dist=np.inf
        u_key=-1
        for u in range(verNum):
            if not visited[u] and dist[u]<min_dist:
                u_key=u
                min_dist=dist[u]

        if u_key==-1:# 已完成查找
            break

        visited[u_key]=1

        for u in range(verNum):
            if np.isfinite(adjM[u_key][u]): # 存在边(边权重不为\math{+\inf})
                if dist[u_key]+adjM[u_key][u]<dist[u]:
                    pred[u]=u_key
                    dist[u]=dist[u_key]+adjM[u_key][u]
    
    print(dist)
    # print(pred)
    # print(visited)

    if with_trace_back:
        trace_back_list=[[] for _ in range(verNum)]
        for target_id in range(verNum):
            if target_id==start_id:
                trace_back_list[target_id]=[target_id]
            else:
                # 从目标 target_id 入手, 根据 pred[] 倒推至 start_id, 然后倒序输出, 即为 start_id 到 target_id 最短路径的路径轨迹
                tmp=target_id
                while not tmp==-1: # pred[start_id]=-1
                    trace_back_list[target_id].append(tmp)
                    tmp=pred[tmp]

                trace_back_list[target_id]=trace_back_list[target_id][::-1]
            print(f'root from id_{start_id} to id_{target_id}: {trace_back_list[target_id]}')
   
    return dist

In [ ]:
import heapq
def dijkstra_Prique(adjM:np.ndarray, start_id:int, with_trace_back=False)->np.ndarray:

    '''
    dijkstra 算法修改版. 输入输出同上, 不再赘述.

    时间复杂度:
        原先通过两层 for 循环寻找最小 dist[u], 复杂度O(verNum^2), 
        这里被替换成了最小堆实现, 复杂度降为 O(verNum * log(verNum)).

        堆的"用武之地":适合处理多次寻找最值的问题, 且每次寻找到一个当前最值后, 
            当前最值被使用而不参与之后轮次的最值寻找
    '''
    
    verNum=adjM.shape[0]
    dist=np.full(shape=verNum, fill_value=np.inf, dtype=np.float32)

    visited=np.zeros(shape=verNum, dtype=np.bool_)
    pred=[-1 for _ in range(verNum)]

    dist[start_id]=0
    # pred[start_id]=-1 已经给所有 pred 赋值为-1, 没必要单独赋值
    
    pq=[]

    heapq.heappush(pq, (dist[start_id], start_id))

    while pq:
        
        current_dist, u_key=heapq.heappop(pq)

        if current_dist>dist[u_key] or visited[u_key]==True: 
        # 跳过无效节点：
        # 1. current_dist > dist[u_key] → 堆中是过时的旧值（已找到更短路径）；
        # 2. visited[u_key] → 节点已处理，最短距离已确定
            continue
        '''
        相比 dijkstra(), 只是把while 内部上面的部分进行了替换. 
        通过一个最小堆来实现最小值的提取和更新.
        其内容架构, 可以参考 Prim算法 的修改: Prim_Prique()函数.
        '''

        visited[u_key]=True
    
        # 更新邻接节点的最短距离
        for u in range(verNum):
            if np.isfinite(adjM[u_key][u]): # 存在边(边权重不为\math{+\inf})
                if dist[u_key]+adjM[u_key][u]<dist[u]:
                    pred[u]=u_key
                    dist[u]=dist[u_key]+adjM[u_key][u]
                    heapq.heappush(pq, (dist[u], u))
        
    print(dist)
    # print(pred)
    # print(visited)

    if with_trace_back:
        trace_back_list=[[] for _ in range(verNum)]
        for target_id in range(verNum):
            if target_id==start_id:
                trace_back_list[target_id]=[target_id]
            else:
                # 从目标 target_id 入手, 根据 pred[] 倒推至 start_id, 然后倒序输出, 即为 start_id 到 target_id 最短路径的路径轨迹
                tmp=target_id
                while not tmp==-1: # pred[start_id]=-1
                    trace_back_list[target_id].append(tmp)
                    tmp=pred[tmp]

                trace_back_list[target_id]=trace_back_list[target_id][::-1]
            print(f'root from id_{start_id} to id_{target_id}: {trace_back_list[target_id]}')
    return dist

![Test Case1](TestCase1.png)

In [ ]:
'''
Test Case1: 如上图, 出处:大二上<集合论与图论>教材&PPT
下计算从下标为0 的结点到其余节点(下标对应法则: 0-v, a-1, b-2, c-3, d-4, w-5)的最短距离
'''
vertexNum=6
edges=[(0, 1, 4), (0, 3, 3), (3, 4, 3), (1, 4, 2), 
       (0, 2, 7), (1, 2, 3), (2, 5, 2), (4, 5, 2)]

adjM=build_adjM(vertexNum, edges, is_directed=True)
dijkstra(adjM, start_id=0, with_trace_back=False)
dijkstra_Prique(adjM, start_id=0, with_trace_back=True)

[0. 4. 7. 3. 6. 8.]
[0. 4. 7. 3. 6. 8.]
root from id_0 to id_0: [0]
root from id_0 to id_1: [0, 1]
root from id_0 to id_2: [0, 2]
root from id_0 to id_3: [0, 3]
root from id_0 to id_4: [0, 3, 4]
root from id_0 to id_5: [0, 3, 4, 5]


array([0., 4., 7., 3., 6., 8.], dtype=float32)

![TestCase2](TestCase2.png)

In [ ]:
'''
Test Case1: 如上图, 出处:大一下<数据结构>PPT
下分别计算从下标为i 的结点到其余节点的最短距离
'''
vertexNum=7
edges=[(0, 1, 10), (0, 2, 2), (1, 4, 1), (2, 3, 2), (3, 4, 4),
       (3, 5, 6), (2, 5, 11), (4, 6, 7), (5, 6, 3)]

adjM=build_adjM(vertexNum, edges, is_directed=False)

for i in range(vertexNum):
    print(f'start_id={i}:')
    dijkstra(adjM, start_id=i, with_trace_back=False)

dijkstra_Prique(adjM, start_id=2, with_trace_back=True)
    

start_id=0:
[ 0.  9.  2.  4.  8. 10. 13.]
start_id=1:
[ 9.  0.  7.  5.  1. 11.  8.]
start_id=2:
[ 2.  7.  0.  2.  6.  8. 11.]
start_id=3:
[4. 5. 2. 0. 4. 6. 9.]
start_id=4:
[ 8.  1.  6.  4.  0. 10.  7.]
start_id=5:
[10. 11.  8.  6. 10.  0.  3.]
start_id=6:
[13.  8. 11.  9.  7.  3.  0.]
[ 2.  7.  0.  2.  6.  8. 11.]
root from id_2 to id_0: [2, 0]
root from id_2 to id_1: [2, 3, 4, 1]
root from id_2 to id_2: [2]
root from id_2 to id_3: [2, 3]
root from id_2 to id_4: [2, 3, 4]
root from id_2 to id_5: [2, 3, 5]
root from id_2 to id_6: [2, 3, 5, 6]


array([ 2.,  7.,  0.,  2.,  6.,  8., 11.], dtype=float32)

In [24]:
vertexNum=3
edges=[(0, 1, 2), (1, 2, 1), (0, 2, 4)]
adjM=build_adjM(vertexNum=vertexNum, edges=edges, is_directed=False)
dijkstra(adjM, start_id=0)


[0. 2. 3.] [-1  0  1] [1 1 1]


array([0., 2., 3.], dtype=float32)